**Algoritmo Genético**:
1. Codifica um algoritmo capaz de fazer uma procura pelo ótimo global.

In [46]:
import numpy as np
import geopy.distance
import random

def give_me_coordinates(start_lat, start_lon, max_delta):
    lat = start_lat + np.random.rand() * max_delta
    lon = start_lon + np.random.rand() * max_delta

    return (lat, lon)

def give_me_distance(start, end, var = 1):
    d = geopy.distance.geodesic(start, end).km
    return d * (1+np.random.randn()/10)

def create_tsp(number_of_cities, start_lat, start_lon, max_delta = 2):

    city_locations = [give_me_coordinates(start_lat, start_lon, max_delta) for _ in range(number_of_cities)]
    total_distances = []
    for start in city_locations:
        distances = []
        for finish in city_locations:
            distances.append(give_me_distance(start, finish))
        total_distances.append(distances)
    return total_distances

In [47]:
# Função para calcular a distância total de uma rota
def objective_function(distances, solution):
    last_city = solution[0]
    total_distance = 0
    for city in solution:
        total_distance += distances[last_city][city]
        last_city = city
    total_distance += distances[last_city][solution[0]]
    return total_distance

In [48]:
# Função para gerar uma população inicial aleatória
def create_initial_population(population_size, num_cities):
    population = []
    for _ in range(population_size):
        individual = list(range(num_cities))
        random.shuffle(individual)
        population.append(individual)
    return population



In [49]:
# Função de seleção por torneio
def tournament_selection(population, fitness, tournament_size):
    selected = []
    for _ in range(len(population)):
        contestants = random.sample(range(len(population)), tournament_size)
        best = min(contestants, key=lambda x: fitness[x])
        selected.append(population[best])
    return selected



In [50]:
# Parâmetros do Algoritmo Genético
population_size = 100
generations = 500
mutation_rate = 0.01
tournament_size = 5

In [51]:
distances = create_tsp(100,50,20)

In [52]:
solucao_candidata = np.random.choice(100, 100, replace=False)
objective_function(distances, solucao_candidata)

9017.449408486576

In [53]:
# Função crossover
def order_crossover(parent1, parent2):
    size = len(parent1)
    child = [-1] * size

    # Escolher dois pontos de divisão aleatórios
    start, end = sorted(random.sample(range(size), 2))

    # Os pontos de corte do parent1 para o child
    child[start:end] = parent1[start:end]

    # Preencher o restante com genes parent2
    pointer = end
    for gene in parent2:
        if gene not in child:
            if pointer >= size:
                pointer = 0
            child[pointer] = gene
            pointer += 1

    return child

In [54]:
# Função mutação
def swap_mutation(individual, mutation_rate):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(individual) - 1)
            individual[i], individual[j] = individual[j], individual[i]
    return individual



In [55]:
# Algoritmo Genético
def genetic_algorithm(distances, population_size, generations, mutation_rate, tournament_size):
    num_cities = len(distances)
    population = create_initial_population(population_size, num_cities)

    # melhor solução global
    best_solution = None
    best_fitness = float('inf')

    for generation in range(generations):
        # fitness para cada individuo
        fitness = [objective_function(distances, individual) for individual in population]

        # melhor solução da geração atual
        current_best_fitness = min(fitness)
        current_best_index = fitness.index(current_best_fitness)
        current_best_solution = population[current_best_index]

        # Atualiza a melhor solução global, se houver
        if current_best_fitness < best_fitness:
            best_solution = current_best_solution.copy()
            best_fitness = current_best_fitness

        # seleção de pais
        parents = tournament_selection(population, fitness, tournament_size)

        # nova população
        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child1 = order_crossover(parent1, parent2)
            child2 = order_crossover(parent2, parent1)
            new_population.append(swap_mutation(child1, mutation_rate))
            new_population.append(swap_mutation(child2, mutation_rate))

        # Substituir a população antiga pela nova
        population = new_population[:population_size]

        # melhor fitness da geração atual
        print(f"Geração {generation + 1}: Melhor Fitness = {best_fitness}")

    # devolver a melhor solução global encontrada
    return best_solution, best_fitness

In [56]:
# Executar o algoritmo genético
best_route, best_distance = genetic_algorithm(distances, population_size, generations, mutation_rate, tournament_size)
print(f"Melhor caminho encontrado: {best_route}")
print(f"Distância total: {best_distance}")

Geração 1: Melhor Fitness = 8802.174256390856
Geração 2: Melhor Fitness = 8506.814252745717
Geração 3: Melhor Fitness = 8352.021735927608
Geração 4: Melhor Fitness = 7964.395465611147
Geração 5: Melhor Fitness = 7867.529440227715
Geração 6: Melhor Fitness = 7333.46147887501
Geração 7: Melhor Fitness = 7333.46147887501
Geração 8: Melhor Fitness = 7153.404519107322
Geração 9: Melhor Fitness = 7153.404519107322
Geração 10: Melhor Fitness = 7082.399481087662
Geração 11: Melhor Fitness = 7032.941900338295
Geração 12: Melhor Fitness = 7002.7465634137425
Geração 13: Melhor Fitness = 6586.385921809894
Geração 14: Melhor Fitness = 6586.385921809894
Geração 15: Melhor Fitness = 6455.912562234773
Geração 16: Melhor Fitness = 6455.912562234773
Geração 17: Melhor Fitness = 6455.912562234773
Geração 18: Melhor Fitness = 6425.384743051122
Geração 19: Melhor Fitness = 6298.9617070397
Geração 20: Melhor Fitness = 6298.9617070397
Geração 21: Melhor Fitness = 6298.9617070397
Geração 22: Melhor Fitness = 

2. Explica a escolha de representação do cromossoma, e a maneira como as duas operações de reprodução foram codificadas:
    * mutação
    * cross-over

Cromossoma representa as soluções possíveis para o problema, neste caso representa o caminho que visita todas as 100 cidades, ou seja contém um conjunto de números inteiros, que depois têm uma correspondência na listagem de distances, no final de cada um dos algoritimos coloquei o resultado do melhor cromossoma encontrado e a respetiva distância.

Mutação, no meu caso, função swap_mutation, implementa uma mutação simples por troca. Para cada gene (cidade) no cromossoma, existe uma probabilidade definida pela mutation_rate de que esse gene seja trocado com outro gene aleatório no cromossoma.

Esta troca introduz aleatoriedade na população e ajuda a explorar novas áreas, evitando que o algoritmo fique preso em ótimos locais. A probabilidade mutation_rate controla o grau de exploração, com valores mais altos levando a mais mudanças e exploração mais ampla.

Crossover, no meu caso, função order_crossover, implementa um crossover de ordem, que preserva a ordem relativa das cidades nos cromossomas pais. O processo é escolher dois pontos de corte aleatórios no cromossoma, copiar a seção entre os pontos de corte do parent1 para o child, e preencher as posições restantes no child com genes do parent2, mantendo a ordem em que aparecem no parent2 e ignora os genes já presentes no filho.
Este tipo de crossover é adequado para o TSP, pois garante que o filho herde informações de ordem dos pais, evitando a criação de rotas inválidas com cidades repetidas.


Genetic algorithm, mas com elitismo, ou seja, conserva sempre a melhor solução de cada geração, teste


In [57]:
def genetic_algorithm_el(distances, population_size, generations, mutation_rate, tournament_size):
    num_cities = len(distances)
    population = create_initial_population(population_size, num_cities)

    # melhor solução global
    best_solution = None
    best_fitness = float('inf')

    for generation in range(generations):
        # fitness para cada individuo
        fitness = [objective_function(distances, individual) for individual in population]

        # melhor solução da geração atual
        current_best_fitness = min(fitness)
        current_best_index = fitness.index(current_best_fitness)
        current_best_solution = population[current_best_index]

        # melhor solução da geração atual
        if current_best_fitness < best_fitness:
            best_solution = current_best_solution.copy()
            best_fitness = current_best_fitness

        # seleção de pais
        parents = tournament_selection(population, fitness, tournament_size)

        # nova população
        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child1 = order_crossover(parent1, parent2)
            child2 = order_crossover(parent2, parent1)
            new_population.append(swap_mutation(child1, mutation_rate))
            new_population.append(swap_mutation(child2, mutation_rate))

        # Substituir a população antiga pela nova
        population = new_population[:population_size]

        # Preservar a melhor solução (elitismo)
        population[0] = best_solution.copy()

        # melhor fitness da geração atual
        print(f"Geração {generation + 1}: Melhor Fitness = {best_fitness}")

    # devolver a melhor solução global encontrada
    return best_solution, best_fitness

In [58]:
# Parâmetros do Algoritmo Genético
population_size = 100
generations = 500
mutation_rate = 0.01
tournament_size = 5

# Executar o algoritmo genético
best_route, best_distance = genetic_algorithm_el(distances, population_size, generations, mutation_rate, tournament_size)
print(f"Melhor caminho encontrado: {best_route}")
print(f"Distância total: {best_distance}")

Geração 1: Melhor Fitness = 8538.62330671984
Geração 2: Melhor Fitness = 7709.222383320555
Geração 3: Melhor Fitness = 7709.222383320555
Geração 4: Melhor Fitness = 7670.534897907216
Geração 5: Melhor Fitness = 7528.531677547881
Geração 6: Melhor Fitness = 6962.750452745939
Geração 7: Melhor Fitness = 6866.988627548922
Geração 8: Melhor Fitness = 6866.988627548922
Geração 9: Melhor Fitness = 6866.988627548922
Geração 10: Melhor Fitness = 6744.753737054203
Geração 11: Melhor Fitness = 6744.753737054203
Geração 12: Melhor Fitness = 6725.804932521646
Geração 13: Melhor Fitness = 6704.960759521165
Geração 14: Melhor Fitness = 6574.528337673201
Geração 15: Melhor Fitness = 6484.8184094998005
Geração 16: Melhor Fitness = 6414.817012190268
Geração 17: Melhor Fitness = 6132.505393993703
Geração 18: Melhor Fitness = 6132.505393993703
Geração 19: Melhor Fitness = 6049.299237257577
Geração 20: Melhor Fitness = 6049.299237257577
Geração 21: Melhor Fitness = 5809.736562526934
Geração 22: Melhor Fit

3. O que acontece ao modelo se a atualização não for aleatória, mas passar a ser sempre aquela em que a distância é a maior? Este modelo tem melhores resultados?

In [59]:
def swap_mutation_longest_distance(individual, distances, mutation_rate):

    if random.random() < mutation_rate:
        # Encontra a maior distancia
        longest_distance = 0
        longest_distance_index = (0, 0)

        for i in range(len(individual)):
            for j in range(i + 1, len(individual)):
                distance = distances[individual[i]][individual[j]]
                if distance > longest_distance:
                    longest_distance = distance
                    longest_distance_index = (i, j)

        # trocar as cidades pelas que formam a maior distância
        i, j = longest_distance_index
        individual[i], individual[j] = individual[j], individual[i]

    return individual

In [65]:
def genetic_algorithm_mutacao_maior_distancia(distances, population_size, generations, mutation_rate, tournament_size):

    num_cities = len(distances)
    population = create_initial_population(population_size, num_cities)

    # melhor solução global
    best_solution = None
    best_fitness = float('inf')

    for generation in range(generations):
        # fitness para cada individuo
        fitness = [objective_function(distances, individual) for individual in population]

        # melhor solução da geração atual
        current_best_fitness = min(fitness)
        current_best_index = fitness.index(current_best_fitness)
        current_best_solution = population[current_best_index]

        # Atualiza a melhor solução global, se houver
        if current_best_fitness < best_fitness:
            best_solution = current_best_solution.copy()
            best_fitness = current_best_fitness

        # seleção de pais
        parents = tournament_selection(population, fitness, tournament_size)

        # nova população
        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child1 = order_crossover(parent1, parent2)
            child2 = order_crossover(parent2, parent1)
            # Changed the function call to swap_mutation_longest_distance
            new_population.append(swap_mutation_longest_distance(child1, distances, mutation_rate))
            new_population.append(swap_mutation_longest_distance(child2, distances, mutation_rate))

        # Substituir a população antiga pela nova
        population = new_population[:population_size]

        # melhor fitness da geração atual
        print(f"Geração {generation + 1}: Melhor Fitness = {best_fitness}")

    # devolver a melhor solução global encontrada
    return best_solution, best_fitness

In [64]:
#comparação de resultados
#1.solução
# Executar o algoritmo genético para obter best_distance da função genetic_algorithm
best_route_ga, best_distance_ga = genetic_algorithm(distances, population_size, generations, mutation_rate, tournament_size)

# Executar o algoritmo genético para obter best_distance da função genetic_algorithm_el
best_route_ga_el, best_distance_ga_el = genetic_algorithm_el(distances, population_size, generations, mutation_rate, tournament_size)

best_route_ga_gd, best_distance_ga_gd = genetic_algorithm_mutacao_maior_distancia(distances, population_size, generations, mutation_rate, tournament_size)

print(f"Distância total (genetic_algorithm): {best_distance_ga}")
print(f"Distância total (genetic_algorithm_el): {best_distance_ga_el}")
print(f"Distância total (genetic_algorithm_gd): {best_distance_ga_gd}")

Geração 1: Melhor Fitness = 7974.874888153055
Geração 2: Melhor Fitness = 7656.476800261665
Geração 3: Melhor Fitness = 7656.476800261665
Geração 4: Melhor Fitness = 7652.609542156284
Geração 5: Melhor Fitness = 7606.211293900491
Geração 6: Melhor Fitness = 7599.662193480863
Geração 7: Melhor Fitness = 7368.86827803215
Geração 8: Melhor Fitness = 7209.681881340997
Geração 9: Melhor Fitness = 6989.17627542319
Geração 10: Melhor Fitness = 6891.5193746353525
Geração 11: Melhor Fitness = 6877.764312032914
Geração 12: Melhor Fitness = 6877.764312032914
Geração 13: Melhor Fitness = 6877.764312032914
Geração 14: Melhor Fitness = 6443.140240795836
Geração 15: Melhor Fitness = 6412.436719188724
Geração 16: Melhor Fitness = 6412.436719188724
Geração 17: Melhor Fitness = 6332.862041741102
Geração 18: Melhor Fitness = 6332.862041741102
Geração 19: Melhor Fitness = 6332.862041741102
Geração 20: Melhor Fitness = 6332.862041741102
Geração 21: Melhor Fitness = 6332.862041741102
Geração 22: Melhor Fitn

Para responder à questão 3, criei um algoritmo (acima) que não usa aleatoriedade, mas sim as distâncias maiores.
Como podemos ver neste caso, de facto, foi o melhor dos 3 algoritmos, mas estamos a falar de um algoritmo que pode correr um grande risco de ficar num ótimo local, provavelmente com mais generations e population, teriamos resultados diferentes.